# DNA/RNA Virus Toolklit

![title](https://www.sciencealert.com/images/2020-03/processed/virus_topic_1024.jpg)

## Import needed Python libraries

In [ ]:
from tqdm import tqdm
import time
import json
import pandas as pd
import numpy as np
import random
from collections import Counter
import os, sys, traceback

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Alphabet import generic_dna

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
from sklearn.feature_selection import RFE
from sklearn import tree

import nltk
from nltk.util import ngrams
from nltk import word_tokenize

plt.style.use('ggplot')

## Structures needed in genome analysis

In [ ]:
# declare valid nucleotides, proteins and their complements, as well as uncertain letters appearing in sequences from NCBI
nucleotides = ["A", "T", "C", "G"]
proteins = ['A','D','E','G','F','L','S','Y','C','W','L','P','H','Q','R','I','M','T','N','K','S','R','V']
complements = {"A": "T", "T": "A", "G": "C", "C": "G"}
uncertainties = ['W', 'U', 'S', 'M', 'K', 'R', 'Y', 'B', 'D', 'H', 'V', 'N', 'Z']
amino_acids = {
    'alanine': 'A', 'arginine': 'R', 'Asparagine': 'N', 'Aspartic Acid': 'D',
    'Cysteine': 'C', 'Glutamic Acid': 'E', 'Glutamine': 'Q',
    'Glycine': 'G', 'Histidine': 'H', 'Isoleucine': 'I', 'Leucine': 'L',
    'Lysine': 'K', 'Methionine': 'M', 'Phenylalanine': 'F', 'Proline': 'P',
    'Serine': 'S', 'Threonine': 'T', 'Tryptophan': 'W', 'Tyrosine': 'Y',
    'Valine': 'V'
}

# using mRNA in lookup, not tRNA anti-codon
start_codons = ["ATG"]
stop_codons = ["TAA", "TAG", "TGA"]
dna_codons = {
    # 'M' - START, '_' - STOP
    "GCT": "A", "GCC": "A", "GCA": "A", "GCG": "A",
    "TGT": "C", "TGC": "C",
    "GAT": "D", "GAC": "D",
    "GAA": "E", "GAG": "E",
    "TTT": "F", "TTC": "F",
    "GGT": "G", "GGC": "G", "GGA": "G", "GGG": "G",
    "CAT": "H", "CAC": "H",
    "ATA": "I", "ATT": "I", "ATC": "I",
    "AAA": "K", "AAG": "K",
    "TTA": "L", "TTG": "L", "CTT": "L", "CTC": "L", "CTA": "L", "CTG": "L",
    "ATG": "M",
    "AAT": "N", "AAC": "N",
    "CCT": "P", "CCC": "P", "CCA": "P", "CCG": "P",
    "CAA": "Q", "CAG": "Q",
    "CGT": "R", "CGC": "R", "CGA": "R", "CGG": "R", "AGA": "R", "AGG": "R",
    "TCT": "S", "TCC": "S", "TCA": "S", "TCG": "S", "AGT": "S", "AGC": "S",
    "ACT": "T", "ACC": "T", "ACA": "T", "ACG": "T",
    "GTT": "V", "GTC": "V", "GTA": "V", "GTG": "V",
    "TGG": "W",
    "TAT": "Y", "TAC": "Y",
    "TAA": "_", "TAG": "_", "TGA": "_"
}

rna_codons = {
    # 'M' - START, '_' - STOP
    "GCU": "A", "GCC": "A", "GCA": "A", "GCG": "A",
    "UGU": "C", "UGC": "C",
    "GAU": "D", "GAC": "D",
    "GAA": "E", "GAG": "E",
    "UUU": "F", "UUC": "F",
    "GGU": "G", "GGC": "G", "GGA": "G", "GGG": "G",
    "CAU": "H", "CAC": "H",
    "AUA": "I", "AUU": "I", "AUC": "I",
    "AAA": "K", "AAG": "K",
    "UUA": "L", "UUG": "L", "CUU": "L", "CUC": "L", "CUA": "L", "CUG": "L",
    "AUG": "M",
    "AAU": "N", "AAC": "N",
    "CCU": "P", "CCC": "P", "CCA": "P", "CCG": "P",
    "CAA": "Q", "CAG": "Q",
    "CGU": "R", "CGC": "R", "CGA": "R", "CGG": "R", "AGA": "R", "AGG": "R",
    "UCU": "S", "UCC": "S", "UCA": "S", "UCG": "S", "AGU": "S", "AGC": "S",
    "ACU": "T", "ACC": "T", "ACA": "T", "ACG": "T",
    "GUU": "V", "GUC": "V", "GUA": "V", "GUG": "V",
    "UGG": "W",
    "UAU": "Y", "UAC": "Y",
    "UAA": "_", "UAG": "_", "UGA": "_"
}

# two test sequences for the below methods, one is COVID, and the other is deformed wing disease (avian), virulent and non-virulent respectively
BT006808_1 = "ATGGCCCTGTGGATGCGCCTCCTGCCCCTGCTGGCGCTGCTGGCCCTCTGGGGACCTGACCCAGCCGCAGCCTTTGTGAACCAACACCTGTGCGGCTCACACCTGGTGGAAGCTCTCTACCTAGTGTGCGGGGAACGAGGCTTCTTCTACACACCCAAGACCCGCCGGGAGGCAGAGGACCTGCAGGTGGGGCAGGTGGAGCTGGGCGGGGGCCCTGGTGCAGGCAGCCTGCAGCCCTTGGCCCTGGAGGGGTCCCTGCAGAAGCGTGGCATTGTGGAACAATGCTGTACCAGCATCTGCTCCCTCTACCAGCTGGAGAACTACTGCAACTAG"
MT324680_1 = "CAAKACTCACTTTCTTCCACAGCAAGTGCACTTGGAAAACTTCAAGATGTGGTCAACCAAAATGCACAAGCTTTAAACACGCTTGTTAAACAACTTAGCTCCAATTTTGGTGCAATTTCAAGTGTTTTAAATGATATCCTTTCACGTCTTGACAAAGTTGAGGCTGAAGTGCAAATTGATAGGTTGATCACAGGCAGACTTCAAAGTTTGCAGACATATGTGACTCAACAATTAATTAGAGCTGCAGAAATCAGAGCTTCTGCTAATCTTGCTGCTACTAAAATGTCAGAGTGTGTACTTGGACAATCAAAAAGAGTTGATTTTTGTGGAAAGGGCTATCATCTTATGTCCTTCCCTCAGTCAGCACCTCATGGTGTAGTCTTCTTGCATGTGACTTATGTCCCTGCACAAGAAAAGAACTTCACAACTGCTCCTGCCATTTGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAA"

## Defined bioinformatic functions

In [ ]:
def translateNucleotides(seq):
    """
    input: string sequence from fasta file or string input
    operation: translates nucleotide sequence into protein sequence, and removes unknown characters
    output: returns a clean translated protein sequence
    """
    
    seq = Seq(seq, generic_dna)
    translated_seq = str(seq.translate()).replace('*', '')
    clean_seq = ''.join([p for p in translated_seq if p in proteins])
    return clean_seq

In [ ]:
def getVirusDf(fasta_data):
    """
    input: fasta data file uploaded by user
    operation: after sequence translation, uses protein analysis \
       and other custom functions to calcuate metrics used in virology research \
    output: returns a dataframe with a feature/column for each engineered feature
    
    """
    
    try:
        ids = [] # accession id, searchable on NCBI
        descs = [] # text description of strain
        aa_percs = [] # ratio of amino acids in strain
        m_weights = [] # molecular weight
        a = [] # aromaticity
        ii = [] # instability index, > 40 means unstable / short half life .instability_index()
        n_seqs = [] # nuleotide sequence
        p_seqs = [] # translated protein sequence

        for record in tqdm(fasta_data):
            print(f'Processing {record.id} - {record.description}')
#             str_seq = cleanPlaceholderNucleotides(str(record.seq))
            str_seq = str(record.seq)
    
            # translated nucleotide string sequence
            p_seq = translateNucleotides(str_seq)
            ids.append(record.id)
            descs.append(record.description)
            
            # GC content
            gc_content = calcGcContent(str_seq)
            
            # protein analysis methods
            analysis = ProteinAnalysis(p_seq)
            aa_percs.append(analysis.get_amino_acids_percent())
            m_weights.append(analysis.molecular_weight())
            ii.append(analysis.instability_index())
            a.append(analysis.aromaticity())
            
            # nucleotide and protein string sequences
            n_seqs.append(str_seq)
            p_seqs.append(str(record.seq.translate()).replace('*', ' '))

        virus_df = pd.DataFrame({
            "ids": ids,
            "description": descs,
            "gc_content": gc_content,
            "amino_acid_percents": aa_percs,
            "molecular_weights": m_weights,
            "aromaticity": a,
            "instability_index": ii,
            "nucleotide_sequence": n_seqs,
            "protein_sequence": p_seqs
        })

        return virus_df
                  
    except Exception as e:
        print('-'*80)
        print(f"Exception in building virus datatframe: {e}")
        traceback.print_exc(file=sys.stdout)
        print('-'*80)

In [ ]:
# test function for taking nucleotide string sequence and translating it to protein sequence
def proteinTranslation(x):
    return str(Seq(x).translate()).replace('*', ' ')

In [ ]:
# manually defined function to return all of the protein analysis components
def proteinAnalysis(s):
    aa_percent, molecular_weight, instability_index, aromaticity = [], [], [], []
    for x in tqdm(s):
        protein_analysis = ProteinAnalysis(x.replace(' ', ''))
        aa_percent.append(protein_analysis.get_amino_acids_percent())
        molecular_weight.append(protein_analysis.molecular_weight())
        instability_index.append(protein_analysis.instability_index())
        aromaticity.append(protein_analysis.aromaticity())
    return aa_percent, molecular_weight, instability_index, aromaticity

In [ ]:
# removes sequences with string length outside of 1.5IQR range
def filterOutliers(df):
    q1 = df.seq_len.quantile(.25)
    q3 = df.seq_len.quantile(.75)
    iqr = q3 - q1
    
    lower_bound = q1 - iqr * 1.5
    upper_bound = q3 + iqr * 1.5
    
    print(f'Lower IQR bound: {lower_bound}, Upper IQR bound: {upper_bound}')
    
    return df[(df.seq_len<=upper_bound) & (df.seq_len>=lower_bound)].copy()

In [ ]:
# used to remove a string buffer for invalid sequences
def removeStringBuffer(seq, i):
    out_seq = seq[:i-i%3] + seq[i+3-i%3:]
    return out_seq.replace('\n', '')

In [ ]:
# used to clean placeholder characters from nucleotide sequence, later not used because only passing valid sequences
def cleanPlaceholderNucleotides(seq):
    k_pos = []
    for i in range(len(seq)):
        if seq[i] not in nucleotides:
            print(f'Found placeholder at {i}th nucleotide')
            k_pos.append(i)
    
    k_seq = seq
    if len(k_pos) > 0:
        for pos in k_pos:
            k_seq = removeStringBuffer(k_seq, pos)

    return k_seq[:(len(k_seq)-len(k_seq)%3)]

In [ ]:
# returns a list of amino acid triples
def buildAminoTriples(s):
    seq_list = []
    for i in range(0, len(s), 3):
        seq_list.append(s[i:i+3])
    return seq_list

In [ ]:
# returns a dictionary with count of all nucleotides in string sequence
def countNucleotides(seq):
#     a_num = seq.count('A')
#     t_num = seq.count('T')
#     c_num = seq.count('C')
#     g_num = seq.count('G')
#     gc_content = (c_num + g_num) / (c_num + g_num + a_num + c_num)
#     nuc_dict = {"A": a_num,
#                "T": t_num,
#                "C": c_num,
#                "G": g_num,
#                "GC": gc_content}
    nuc_dict = dict(Counter(seq))
    return nuc_dict

In [ ]:
# validates if a sequence contains only valid DNA nucleotides (A, C, T, G)
def validateString(seq):
    tmp_seq = seq.upper()
    for nuc in tmp_seq:
        if nuc not in nucleotides:
            return False
    return True

In [ ]:
# generates a random sequence of DNA nucleotides
def generateRandomString(num_nucs):
    rand_seq = ''.join([random.choice(nucleotides) for nuc in range(num_nucs)])
    return rand_seq

In [ ]:
# transcribes a nucleotide sequence from DNA to RNA (aka swapping T for U)
def transcription(seq):
    return seq.replace("T", "U")

In [ ]:
# returns the reverse complement of the nucleotide sequence
def reverseComplement(seq):
    return ''.join([complements[nuc] for nuc in seq])[::-1]

In [ ]:
# calculates the GC content of the nucleotide sequence
def calcGcContent(seq):
    return round((seq.count("C") + seq.count("G")) / len(seq) * 100, 3)

In [ ]:
# calculates the GC content of a subsequence, default range 20
def subseqGcContent(seq, k=20):
    res = []
    for i in range(0, len(seq) - k + 1, k):
        subseq = seq[i:i + k]
        res.append(calcGcContent(subseq))
    return res

In [ ]:
# translates DNA or RNA nucleotide sequence to proteins by looking up codon dictionary values above
def translateSequence(seq, seq_type='dna', init_pos=0):
    if seq_type == 'dna':
        return [dna_codons[seq[pos:pos+3]] for pos in range(init_pos, len(seq) -2, 3)]
    elif seq_type == 'rna':
        return [rna_condons[seq[pos:pos+3]] for pos in range(init_pos, len(seq) -2, 3)]
    else:
        return ['invalid genome type provided']

In [ ]:
# identifies the frequency of codons in an amino acid string
def codonUsage(seq, aminoacid):
    codon_list = []
    for i in range(0, len(seq) - 2, 3):
        if dna_codons[seq[i:i + 3]] == aminoacid:
            codon_list.append(seq[i:i + 3])
            
    codon_frequencies = dict(Counter(codon_list))
    total_weight = sum(codon_frequencies.values())
    for seq in codon_frequencies:
        codon_frequencies[seq] = round(codon_frequencies[seq] / total_weight, 2)
    return codon_frequencies

In [ ]:
# creates reading frames from protein sequence
def generateReadingFrames(seq):
    frames = []
    frames.append(translateSequence(seq, 'dna', 0))
    frames.append(translateSequence(seq, 'dna', 1))
    frames.append(translateSequence(seq, 'dna', 2))
    frames.append(translateSequence(reverseComplement(seq), 'dna', 0))
    frames.append(translateSequence(reverseComplement(seq), 'dna', 1))
    frames.append(translateSequence(reverseComplement(seq), 'dna', 2))
    return frames

In [ ]:
# extracts protein sequence from the amino acid string, uses codon triplets
def proteinsFromReadingFrame(amino_acids):
    current_protein = []
    proteins = []
    
    for amino_acid in amino_acids:
        if amino_acid == "_":
            if current_protein:
                for protein in current_protein:
                    proteins.append(protein)
                current_protein = []
        else:
            if amino_acid == "M":
                current_protein.append("")
            for i in range(len(current_protein)):
                current_protein[i] += amino_acid
    return proteins

In [ ]:
# aggregates proteins from reading frames into a complete protein sequence
def aggregateProteinsFromReadingFrames(seq, start_pos=0, end_pos=0, ordered=False):
    if end_pos > start_pos:
        reading_frames = generateReadingFrames(seq[start_pos: end_pos])
    else:
        reading_frames = generateReadingFrames(seq)
    
    result = []
    for frame in reading_frames:
        proteins = proteinsFromReadingFrame(frame)
        for protein in proteins:
            result.append(protein)
            
    if ordered:
        return sorted(result, key=len, reverse=True)
    return result

## Import CSV files of valid sequences

After running the sequence_splitter.ipynb, two output CSV files were created.  Starting with the cell below, these two files of valid and complete virus DNA sequences will be imported into this environment to run analysis.  The source data for this analysis was obtained by downloading nucleotide sequences in FASTA file format from the National Institute of Health's NCBI Virus database.  For example, here is a link that shows where data for the complete COVID genome sequences were obtained:

https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=Severe%20acute%20respiratory%20syndrome%20coronavirus%202,%20taxid:2697049&SourceDB_s=GenBank&Completeness_s=complete

In [ ]:
t1 = time.time()
non_vir_df = pd.read_csv('valid_non_virulent_sequences.csv') # archive of valid (only ATCG) non-virulent sequences
vir_df = pd.read_csv('valid_virulent_sequences.csv') # archive of valid (only ATCG) non-virulent sequences

print(f'Took {time.time() - t1} seconds to upload data files')

In [ ]:
# give virulent df a categorical class, change to category type and add column for sequence length
vir_df['class'] = 'virulent'
vir_df['class'] = vir_df['class'].astype('category')
vir_df['seq_len'] = vir_df['seq_str'].apply(lambda x: len(x))

# give non-virulent df a categorical class, change to category type and add column for sequence length
non_vir_df['class'] = 'non-virulent'
non_vir_df['class'] = non_vir_df['class'].astype('category')
non_vir_df['seq_len'] = non_vir_df['seq_str'].apply(lambda x: len(x))

Inspecting the imported files, we can see there is a roughly 3:1 split between virulent and non-virulent genomes.

In [ ]:
non_vir_df.info()

In [ ]:
vir_df.info()

## View distribution of length of DNA/RNA sequences

In [ ]:
non_vir_df.seq_len.plot(kind='box', rot=90, figsize=(5,5))

In [ ]:
non_vir_df[non_vir_df['seq_len']>=100000]['seq_name'].value_counts()

In [ ]:
vir_df.seq_len.plot(kind='box', rot=90, figsize=(5,5))

In [ ]:
vir_df[vir_df['seq_len']>=175000]['seq_name'].value_counts()

From observing the box plots for the two classes of viruses, there are clearly outliers present in the data.  For virulent sequences, we will be later filtering around DNA sequence length of 50,000 nucleotides, and same for non-virulent.

## Sample and concatenate two dataframes

In order to validate the outliers found above, we will take a sample from each dataframe, and then merge/concat the two objects into a single one to show the box plot again and test if the outliers are still the same.  We found out that there is still a thredshold around 25,000 and around 50,000.  For the virulent strains, those above 175,000 are Small Pox, while uncertain which species of virus for the non-virulent becuase there are over 300 of animal or plant viruses.

In [ ]:
vir_df_sample = vir_df.sample(n=10000, weights='seq_len', random_state=42)
non_vir_df_sample = non_vir_df.sample(n=10000, weights='seq_len', random_state=42)

In [ ]:
sample_df = pd.concat([vir_df_sample, non_vir_df_sample])
sample_df.reset_index(drop=True, inplace=True)

In [ ]:
sample_df.head()

In [ ]:
sample_df.seq_len.plot(kind='box')

In [ ]:
merged_df = pd.concat([vir_df, non_vir_df])

In [ ]:
# filtered_df = filterOutliers(merged_df)
filtered_df = merged_df[merged_df.seq_len<=50000]
filtered_df.reset_index(drop=True, inplace=True)
filtered_df.seq_len.plot(kind='box')

## Reconstruct common protein analysis

These next few cells are the first iteration on how we performed the protein analysis.  They take a few minutes to run each because there are over 40,000 sequences, of length in the hundreds to 50,000 nucleotides, so the conversion process is computationally intensive.

#### First, translate the DNA nucleotide sequences to proteins

In [ ]:
t1 = time.time()
filtered_df['p_seq'] = filtered_df['seq_str'].apply(lambda x: proteinTranslation(x))
print(f'Took {time.time() - t1} seconds to translate nucleotides to proteins')

#### Calculates the GC content from DNA sequence of nucleotides

In [ ]:
filtered_df['gc_content'] = filtered_df['seq_str'].apply(lambda x: calcGcContent(x))

#### Renaming the sequence columns to be more user friendly

In [ ]:
filtered_df.rename(columns={'p_seq': 'protein_sequence', 'seq_str': 'nucleotide_sequence'}, inplace=True)

#### For each sequence, run a protein analysis on the protein sequence to obtain metrics/features

In [ ]:
aa, mw, a, ii = np.apply_along_axis(proteinAnalysis, 0, filtered_df['protein_sequence'].values)
new_cols = ['amino_acid_percents','molecular_weights','aromaticity','instability_index']
filtered_df[new_cols] = pd.DataFrame({'amino_acid_percents': aa, 
                                      'molecular_weights': mw, 
                                      'aromaticity': a, 
                                      'instability_index': ii})

In [ ]:
filtered_df.head()

#### Clean the sequence ID column as they contain suffixes which won't be able to join metadata files

In [ ]:
filtered_df['seq_id'] = filtered_df['seq_id'].apply(lambda x: x.replace('.1', '').replace('.2', '').replace('.3', '').replace('.4', '').replace('.5', ''))

#### Convert numeric columns to float data type

In [ ]:
num_cols = ['molecular_weights', 'aromaticity', 'instability_index']
filtered_df[num_cols] = filtered_df[num_cols].astype('float')

In [ ]:
filtered_df.describe()

In [ ]:
filtered_df.set_index('seq_id', inplace=True)

### Import metadata for pathogenic and nonpathogenic sequences

In addition to using the raw sequence data, we thought there might be something useful in the metadata of the tables in the NCBI Virus database.  So here we import the two files of same sequences (virulent and non-virulent) and then join the sequence data to the metadata on the index, which is also called the Accession ID.  Since the suffix to sequence ID was removed, the join will now be successful.

In [ ]:
vir_meta_df = pd.read_csv('pathogenic_sequences_table.csv')
non_vir_meta_df = pd.read_csv('non_pathogenic_sequences_table.csv')

In [ ]:
vir_meta_df.set_index('Accession', inplace=True)
vir_meta_df.info()

In [ ]:
non_vir_meta_df.set_index('Accession', inplace=True)
non_vir_meta_df.info()

In [ ]:
meta_df = pd.concat([vir_meta_df, non_vir_meta_df])

In [ ]:
filtered_df = filtered_df.merge(meta_df, how='left', left_index=True, right_index=True)

In [ ]:
filtered_df.head()

### Save point #1

#### Saving the state of this dataframe in CSV so that we can resume the analysis below without performing the above tasks over again, since they were somewhat computationally intensive to translate to proteins and run protein analysis on 40,000 sequences.

In [ ]:
filtered_df.to_csv('virus_analysis.csv')

### Inspect feature distributions

Here is a save point where we can resume analysis in the notebook withouth having to repeat the protein translation and analysis steps.  This will just import the saved state as a new dataframe object. 

In [ ]:
analysis_df = pd.read_csv('virus_analysis.csv')

In [ ]:
analysis_df.isnull().sum()

In [ ]:
analysis_df.drop('Isolation_Source', axis=1, inplace=True)

#### Here is a list of all the unique virus names found in the metadata for sequences

In [ ]:
virus_names = analysis_df.sort_values(by='Species').Species.unique().tolist()
virus_names

#### Making the species names to lower case text format

In [ ]:
analysis_df['Species'] = analysis_df['Species'].apply(lambda x: x.lower())

#### Here is the count of viruses by family name under genus

In [ ]:
virus_families = analysis_df.Family.unique().tolist()
analysis_df.Family.value_counts()

#### Similarly, we can count the number of virulent vs. non-virulent in each family using crosstab

In [ ]:
pd.crosstab(analysis_df['Family'], analysis_df['class']).sort_values(by='virulent', ascending=False)

#### Defining a function to iteratively loop through each virus name specified to see the sequence length distributions

In [ ]:
def plotSequenceLenHistogram(df, bins, strain):
    df[(df['Species'].str.contains(strain))]['seq_len'].hist(bins=bins)
    plt.title(f'{strain} Protein Length Distribution')
    plt.show()

In [ ]:
virus_names = ['ebola', 'hanta', 'severe acute respiratory', 'influenza', 'rabies', 'dengue', 'hepatitis', 'hepacivirus', 'rota', 'chikungunya', 'west nile', 'zika', 'measles', 'middle east']

In [ ]:
for name in virus_names:
    plotSequenceLenHistogram(analysis_df, 50, name)

#### Creating a pairplot in seaborn to see if there are any correlations between the protein analysis metrics

In [ ]:
sns.pairplot(data=analysis_df[['seq_len', 'gc_content', 'molecular_weights', 'aromaticity', 'instability_index', 'class']], hue='class')
plt.show()

THere aren't any clear correlations amonths these feature pairs, although sequence length does appear to have a role in distinguishing the two classes for molecular weights, aromaticity and instability index.

#### Hisogram of sequence length for virulent virus species

In [ ]:
analysis_df[(analysis_df['class']=='virulent')].seq_len.hist(bins=50)
#  & (analysis_df['seq_desc'].str.contains('complete')) & (analysis_df['seq_len']>= 10000)
plt.title('virulent')
plt.show()

#### Hisogram of sequence length for non-virulent virus species

In [ ]:
analysis_df[(analysis_df['class']=='non-virulent')].seq_len.hist(bins=50) 
#  & (analysis_df['seq_desc'].str.contains('complete')) & (analysis_df['seq_len'] <= 80000) & (analysis_df['seq_len']>= 10000)
plt.title('non-virulent')
plt.show()

As we can see in the two histograms above, it is lear that the sequence length for non-virulent strains has a near normal distribution between 0 to 10,000 length with some "outliers" above 40,000.  The virulent strains however are very multimodal, and more sparse, or not consistently clustered in the same length range.

This cell below can be ignored because it was used to filter strains to complete and to specific sequence length ranges based on class, but we figured this might inject some bias in the sample used to train models below.  The viruses should be complete based on the downloaded files, and the entire dataset has already removed outliers of length >= 50,000 nucleotides.

#### Showing where the virus sequences obtained from the database originate

In [ ]:
analysis_df['Country'] = analysis_df['Geo_Location'].apply(lambda x: str(x).split(':')[0])
analysis_df['Country'] = analysis_df['Country'].replace(np.nan, 'Unknown')

In [ ]:
country_counts = analysis_df['Country'].value_counts().reset_index()
country_counts.columns = ['Country', 'Virus_Count']

In [ ]:
f, ax = plt.subplots(figsize=(15, 8))
sns.barplot(x='Country', y='Virus_Count', data=country_counts.head(50), ax=ax)
plt.title('Count of virus sequences in dataset by country')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# analysis_df = pd.concat([
#     filtered_df[(filtered_df['class']=='virulent') & (filtered_df['seq_desc'].str.contains('complete')) & (filtered_df['seq_len']>= 10000)],
#     filtered_df[(filtered_df['class']=='non-virulent') & (filtered_df['seq_desc'].str.contains('complete')) & (filtered_df['seq_len'] <= 80000) & (filtered_df['seq_len']>= 10000)]
# ])

#### Replacing single quote with double quotes in amino acid percents column so that json object can be loaded

In [ ]:
analysis_df['amino_acid_percents'] = analysis_df['amino_acid_percents'].apply(lambda x: json.loads(x.replace("'", '"')))

#### Create a column name for each amino acid to be parsed from dictionary

In [ ]:
aa_list = list(amino_acids.values())
aa_cols = ['amino_acid_' + aa for aa in aa_list]

#### Parse amino acid percent dictionary into a series, and perform lambda column renaming

In [ ]:
amino_acids_percent_df = analysis_df['amino_acid_percents'].apply(pd.Series)
amino_acids_percent_df = amino_acids_percent_df.rename(columns = lambda x : 'amino_acid_' + str(x))

#### Correlation heatmap of amino acid consistency

In [ ]:
#Showing mutual information as heatmap with values
corr = amino_acids_percent_df.corr()
plt.figure(figsize = (16,16))
ax = sns.heatmap(corr,annot=True, linewidths=.5);

From this heatmap, we can see that multiple amino acid pairs have a high correlation.  For example, amino acid F and Y, as well as amino acids D with M and K, all have over .50 correlation R^2 value.  We're not subject matters on amino acids so not sure what this means.

In [ ]:
# script for plotting kde with scatter obtained from seaborn documentation
# https://seaborn.pydata.org/tutorial/distributions.html
g = sns.jointplot(x="amino_acid_E", y="amino_acid_Y", data=amino_acids_percent_df, kind="kde", color="m")
g.plot_joint(plt.scatter, c="w", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$X$", "$Y$");

Clearly there are too many dense overlapping data points for this to be meaningful.

#### KDE scattergram of comparing amino acid E and Y together, since this pair has .55 correlation

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
sns.kdeplot(amino_acids_percent_df["amino_acid_E"], amino_acids_percent_df["amino_acid_Y"], ax=ax)
sns.rugplot(amino_acids_percent_df["amino_acid_E"], color="g", ax=ax)
sns.rugplot(amino_acids_percent_df["amino_acid_Y"], vertical=True, ax=ax)
plt.title('KDE Plot of relationship between Amino Acids E and Y');

The KDE plot helps to see the density and maybe some of the underlying correlation between these two features.

#### Merging/concatenating the analyais dataframe with the amino acids

In [ ]:
analysis_df = pd.concat([analysis_df, amino_acids_percent_df], axis=1)

In [ ]:
analysis_df.columns

## Build TFIDF vectors to test for clustering in protein similarity 

We wanted to see if there was a way to cluster the viruses using the genome sequence data, and not the protein analysis metrics first.  This was an initial attempted at unsupervised learning with this dataset.  When you see a protein sequence, it literally looks like a long string of letters from the alphabet.  Each letter represents an amino acid, and a chain of amino acids in between start and stop codons (triplets of nucleotides) creates a protein, but when the proteins are separated with spaces, it kind of looks like a sentence of words.  This gave me the idea of vectorizing the proteins using the TFIDF algorithm, and then performing cluster and similarity analysis with the "syntax" of relevant protein subsequences.

In [ ]:
protein_corpus = analysis_df.protein_sequence.values

#### Build the TFIDF vectorizer model and transform the corpus of proteins (array of protein sequences) into vectors.

In [ ]:
t1 = time.time()

vectorizer = TfidfVectorizer(ngram_range=(1, 10), 
                             analyzer='word', norm='l2', 
                             max_features=5000, lowercase=False,
                             min_df=.01, max_df=.90)
protein_vectors = vectorizer.fit_transform(protein_corpus)

t2 = time.time()

print(f'Took {t2-t1:.3} seconds to build TFIDF vector')

In [ ]:
protein_vectors.shape

In [ ]:
protein_vectors.toarray()[0:5]

#### Here is the complete list of feature names, or protein subsequences from the vectorized corpus

In [ ]:
vectorizer.get_feature_names()

#### Create a new dataframe object with selected columns and merge with TFIDF protein vectors

In [ ]:
ids = analysis_df.reset_index().seq_id.values
classes = analysis_df['class'].values
species = analysis_df['Species'].values
family = analysis_df['Family'].values
country = analysis_df['Country'].values
amino_acids = analysis_df[aa_cols].values

In [ ]:
num_cols = ['seq_len', 'gc_content', 'molecular_weights', 'aromaticity', 'instability_index']
numeric = analysis_df[num_cols].values

scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(numeric)

In [ ]:
vector_df = pd.DataFrame(protein_vectors.toarray())
vector_df['ids'] = ids
vector_df['class'] = classes
vector_df['species'] = species
vector_df['family'] = family
vector_df['country'] = country
vector_df[aa_cols] = pd.DataFrame(amino_acids)
vector_df[num_cols] = pd.DataFrame(numeric_scaled)
vector_df.set_index('ids', inplace=True)

In [ ]:
vector_df.groupby(['class','family']).size()

### Save point #2 post transformation to TFIDF vector, and merging with analysis dataframe

In [ ]:
# vector_df.to_csv('analysis_vectors.csv')

If continuing from running the code above, there is no need to run the next cell to create vector_df from the CSV file, as it's already in memory.  This save point is just to save time and jumping back into the analysis at a different part of the notebook workflow.

In [ ]:
# vector_df = pd.read_csv('analysis_vectors.csv')

In [ ]:
vector_df.head()

In [ ]:
vec_cols = list(np.arange(1000))
num_cols = ['seq_len', 'gc_content', 'molecular_weights', 'aromaticity', 'instability_index']
cat_cols = ['species', 'family', 'country']

#### Output CSV of aggregation on class family and species by average of each TFIDF vector feature

In [ ]:
# vector_df.groupby(by=['class', 'family', 'species'])[vec_cols].mean().to_csv('class_vec_mean.csv')

### Dimensionality reduction using SVD (Singular Value Decomposition)

In order to perform some unsupervised clustering on the virus dataset, we transformed the data using truncated SVD method.  This allowed us to reduce the dimensionality of the dataset down to just 5 features.  Although, this is still too many features to be represented on a 2D or 3D visualization.

In [ ]:
svd_df = vector_df.drop(['class', 'species', 'family', 'country'], axis=1)
svd_df.set_index('ids', inplace=True)

In [ ]:
svd_df.head()

In [ ]:
# svd_features = np.concatenate((protein_vectors.toarray(), vector_df[num_cols].values), axis=1) # with numeric features
svd_features = protein_vectors.toarray() # without numeric features

In [ ]:
# https://chrisalbon.com/machine_learning/feature_engineering/select_best_number_of_components_in_tsvd/
# Create a function
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [ ]:
svd_df.shape

In [ ]:
svd_df.info()

In [ ]:
tsvd = TruncatedSVD(n_components=30)
X_tsvd = tsvd.fit(svd_df)

In [ ]:
tsvd_var_ratios = tsvd.explained_variance_ratio_

In [ ]:
select_n_components(tsvd_var_ratios, 0.95)

In [ ]:
svd = TruncatedSVD(n_components=6, random_state=42)
svd_matrix = svd.fit_transform(svd_df)

In [ ]:
tsvd_range = np.arange(1,31)

In [ ]:
plt.plot(tsvd_range, tsvd_var_ratios)
plt.xlim(0,25)
plt.title('Ratio of Explained Variance for each k components in SVD')
plt.show()

This should tell us that the ideal number of SVD components based on explained variance is 5.

In [ ]:
for i in range(0,6):
    svd_col = 'svd_' + str(i+1)
    vector_df[svd_col] = svd_matrix[:, i]

#### Validation that the SVD features are added to the vector dataframe

In [ ]:
vector_df.head()

### Clustering using Kmeans

We can also use an algorithm like Kmeans to see if there is potential for clustering the viruses based on the engineered features from protein vectors and protein analysis metrics.

In [ ]:
ssd = []

for i in range(1, 40):
    print(f'Building kmeans with {i+1} clusters')
    kmeans = KMeans(n_clusters=i+1)
    kmeans.fit(svd_matrix)
    print(f'Current sum of squared distance: {kmeans.inertia_:.2}')
    ssd.append(kmeans.inertia_)
    print('-'*100)

In [ ]:
#Plotting to find the number of clusters
plt.plot(range(1,40), ssd,'bx-')
plt.xlabel("No. of clusters")
plt.ylabel("Sum of Squared Distance") #sum of squares within clusters
plt.title("The Elbow Method showing the optimal Kmeans clusters\n")
plt.show()

Using the elbow method for KMeans inertia (sum of squared distance between instances) we can see that even though there is a distinguishable change in slope between K = 10 and K = 20, the sum of squared distance is still at a very high scale.  Usually the SSD would be significantly lower.  At this xcale, the required number of clusters to reduce the SSD to a reasonable level means we would need maybe hundreds of clusters.  Therefore this method won't yield good results.

In [ ]:
kmeans = KMeans(n_clusters=20)
kmeans.fit(vector_df.drop(['ids','class','species','family','country'], axis=1))

In [ ]:
vector_df['cluster'] = kmeans.predict(vector_df.drop(['ids','class','species','family','country'], axis=1))

In [ ]:
k_max = 40
k_range = range(1, k_max+1)

# Cluster data into k=1..k_max clusters.
ests = [KMeans(n_clusters=k).fit(svd_matrix) for k in k_range]

In [ ]:
from scipy.spatial.distance import cdist, pdist

In [ ]:
def get_dists(X, centroids):
    """Calculate distance of each instance from nearest centroid."""
    dists = [cdist(X, center, 'euclidean') for center in centroids]
    dist = [np.min(dist, axis=1) for dist in dists]
    c_idx = [np.argmin(dist, axis=1) for dist in dists]
    
    return dist, c_idx


def get_metrics(X, dist):
    """Cluster "goodness" metrics."""
    total_within_ss = [sum(d**2) for d in dist]  # total within-cluster sum of squares
    total_ss = sum(pdist(X)**2) / X.shape[0]     # total sum of squares
    between_ss = total_ss - total_within_ss      # between-cluster sum of squares
    
    return total_ss, between_ss


def plot_elbow(k_range, total_ss, between_ss, n_true=None):
    """Plot elbow curve."""
    fig = plt.figure(figsize=(10, 8))

    var_exps = (between_ss / total_ss) * 100

    ax = fig.add_subplot(111)
    ax.plot(k_range, var_exps, 'b*-')

    if n_true is not None:
        # Add marker for ground truth.
        kwargs = {'marker': 'o', 'markersize': 12, 'markeredgewidth': 2,
                  'markeredgecolor': 'r', 'markerfacecolor': 'None'}
        k_true = k_range[n_true-1]
        k_true_var_exp = (between_ss[n_true-1] / total_ss) * 100
        ax.plot(k_true, k_true_var_exp, **kwargs)

    ax.set_xlim((0, k_range[-1]))
    ax.set_ylim((0, 100))
    ax.set_xticks(k_range)

    plt.grid(True)
    plt.xlabel("Number of Clusters")
    plt.ylabel("Variance Explained (%)")
    plt.title("k-Means Clustering Elbow Plot")

    plt.show()

In [ ]:
centroids = [est.cluster_centers_ for est in ests]

#### NOTE: calculating the distance from centroids might take a few minutes

In [ ]:
dist, c_idx = get_dists(svd_matrix, centroids)
total_ss, between_ss = get_metrics(svd_matrix, dist)

In [ ]:
plot_elbow(k_range, total_ss, between_ss, n_true=20)

#### Scatterplot of truncated SVD components with cluster, class and species, etc.

In [ ]:
import plotly.express as px
fig = px.scatter_3d(vector_df.reset_index(), x="svd_1", y="svd_2", z='svd_3', color="class", opacity=.25,
                 hover_data=['class', 'ids', 'species', 'family', 'country', 'cluster'], color_continuous_scale=px.colors.sequential.Viridis)
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter_3d(vector_df.reset_index(), x="svd_1", y="svd_2", z='svd_3', color="cluster", opacity=.25,
                 hover_data=['class', 'ids', 'species', 'family', 'country', 'cluster'], color_continuous_scale=px.colors.sequential.Viridis)
fig.show()

Surprisingly with 20 custers, the virus sequences are separated pretty well.  For exampe, Middle East Respiratory Syndrome and Severe Accute Respiratory Syndrom are near eachother, but labeled with two different clusters.  So even in a small dimensional area, the clustering is pretty good.

### Attempt at Dendrogram for Species

In [ ]:
def plot_dendrogram(model, **kwargs):
    children = model.children_
    dist = np.arange(children.shape[0])+1
    n_obs = np.arange(2, children.shape[0]+2)
    linkage_matrix = np.column_stack([children, dist, n_obs])
    dendrogram(linkage_matrix.astype(float), **kwargs)

In [ ]:
n_family = vector_df['family'].nunique()
n_family

In [ ]:
X_vect = vector_df[['seq_len', 'gc_content', 'molecular_weights', 'aromaticity', 'instability_index','cluster','class']].copy()
classes = X_vect.pop('class')

In [ ]:
# model = AgglomerativeClustering(n_clusters=n_family)
# model = model.fit(X_vect)

In [ ]:
# figure = plt.figure(figsize=(12, 20))
# labels = vector_df['species'].values
# plt.title('Hierarchical Clustering Dendrogram of Virus RNA')
# plot_dendrogram(model, labels=labels, orientation='right', leaf_font_size=9)
# plt.savefig('species_svd_dendrogram.png')
# plt.show()

We were unsuccessful at creating a dendrogram with the full dataset or even a scaled down one using a selection of the SVD components generated from the protein vectors with the protein analysis metrics.  The agglomerative clustering fitting and dendrogram plotting process takes too long, so we've had to cancel the process many times.

### Cluster map from Kmeans cluster and protein analysis features

#### NOTE: Running the clustermap visualization might take a few minutes

In [ ]:
sns.set(color_codes=True)
lut = dict(zip(classes.unique(), "rbg"))
row_colors = classes.map(lut)
g = sns.clustermap(X_vect, row_colors=row_colors)
plt.title('Dendrogram Clustermap of Protein metrics with Kmeans Cluster')
plt.show()

## Training Predictive Models

#### Use simple OLS linear regression and LogisticRegression as baseline performance

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
vec_cols = list(np.arange(5000))
vec_cols_str = [str(col) for col in vec_cols]
vec_col_str = ' + '.join(vec_cols_str)

In [ ]:
svd_cols = ['svd_' + str(i+1) for i in np.arange(6)]

In [ ]:
svd_cols

In [ ]:
target_col = 'class'

In [ ]:
x = np.matrix(vector_df[vec_cols_str].values)
y = np.array(vector_df[target_col].values).reshape(-1, 1)

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
y_bin = []
for i in range(len(y)):
    if y[i] == 'virulent':
        y_bin.append(1)
    else:
        y_bin.append(0)

#### NOTE: running the Ordinary Least Squares (OLS) model takes a few minutes to print summary

In [ ]:
model = sm.OLS(y_bin, x).fit()
print(model.summary())

In [ ]:
model.params

In [ ]:
pval_df = pd.DataFrame({'vector': vec_cols, 'pval': model.pvalues, 'coef': model.params})
pval_df['proteins'] = vectorizer.get_feature_names()
# pval_df[pval_df['pval'] <= .05].to_csv('protein_importance.csv', index=False)
pval_df.to_csv('protein_importance_unfiltered.csv', index=False)
pval_list = pval_df[pval_df['pval'] <= .05]['vector'].tolist()

#### Index Positions of 25 out of 102 most relevant/important proteins

In [ ]:
pval_list[0:25]

Based on the OLS model, there are 102 protein vectors which have a statistically significant relationship with the predictive outcome for virulent class.  However, OLS isn't optimized for a binary nor multiclass classification problem.  However, this might still provide useful insight into the feature importance for training models more appropriate for classification.  Most of the relevant protein vectors are very short, only 2-3 protein characters long, however some are longer.  For example:

EAVGINWSVHQHHATFSPLPNHLMFMSYCSSLQAVPWVALGHGH

This is a protein associated with Hepatits B virus!

In [ ]:
model_df = analysis_df.copy()

In [ ]:
# model_df.to_csv('model_data.csv')

### Save point #3

In [ ]:
# model_df = pd.read_csv('model_data.csv')

In [ ]:
vec_cols = list(np.arange(0,5000)) # vec_cols + 
cols_to_drop = ['seq_id','seq_name','seq_desc','nucleotide_sequence','protein_sequence','class','Country','Genus','Species','Family','Length','Nuc_Completeness','Geo_Location','Host','Collection_Date','Release_Date','amino_acid_percents']

In [ ]:
model_df['class'] = model_df['class'] == 'virulent'

In [ ]:
model_df['class'].value_counts()

In [ ]:
model_df.head()

In [ ]:
model_df.drop(cols_to_drop, axis=1).info()

In [ ]:
cols_to_drop_w_class = [col for col in cols_to_drop if col != 'class']

#### Correlation heatmap of model features

In [ ]:
model_corr = model_df.drop(cols_to_drop_w_class, axis=1).corr()
plt.figure(figsize=(15,15))
sns.heatmap(model_corr, annot=True);

### Splitting dataframe of features for predictive model by random 70/30

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(model_df.drop(cols_to_drop, axis=1), model_df['class'], test_size=0.3, random_state=42)

In [ ]:
y_test.value_counts()

### Building a Guassian Naive Bayes Classifier

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)

In [ ]:
y_train.head()

#### Confusion Matrix

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred_nb, labels=[False, True]), annot=True)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.show()

Unlike the Extra Trees classifier, the Naive Bayes model creates over 5,000 false negatives and over 600 false positives.

#### Accuracy Score

In [ ]:
gnb.score(X_test, y_test)

#### Classification Report

In [ ]:
print(classification_report(y_test, y_pred_nb, labels=[False, True]))

### Building a Logistic Regression Classifier

In [ ]:
lrc = LogisticRegression()
lrc.fit(X_train, y_train)
y_pred_lr = lrc.predict(X_test)

#### Accuracy Score

In [ ]:
lrc.score(X_test, y_test)

#### Confusion Matrix

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred_lr), annot=True)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.show()

#### Classification Report

In [ ]:
print(classification_report(y_test, y_pred_lr, labels=[False, True]))

### Selection Relevant Features for Logistic Regression

Since the logistic regression model performed well, but not the best (significantly better than NB), we wanted to try using a feature selection model to narrow the feature set to optimize the logistic regression model accuracy.

In [ ]:
selector = RFE(lrc, n_features_to_select=1)
selector = selector.fit(X_train, y_train)

In [ ]:
selector.ranking_

In [ ]:
lrc.coef_

In [ ]:
Xy = pd.concat([X_train, y_train], axis=1)

In [ ]:
Xy['class_label'] = Xy['class'].apply(lambda x: 'virulent' if x else 'non-virulent')

#### Correlation heatmap of train data

In [ ]:
Xy_corr = Xy.corr()
plt.figure(figsize=(10,10))
sns.heatmap(Xy_corr);

As we can see from the correlation heatmap of just the training data, only a few features have a higher correlation coefficient with the target variable 'class'.

#### Creating a mask of features to keep based on logistic regression classifier coefficient

In [ ]:
mask = np.round(lrc.coef_, decimals=2)>0

In [ ]:
cols_to_keep = []

In [ ]:
for ii, val in enumerate(mask[0]):
    print(ii, val)
    if val == True:
        cols_to_keep.append(X_train.columns.tolist()[ii])

In [ ]:
cols_to_keep

In [ ]:
X_train[cols_to_keep].head()

#### Trying to split the data on features kept from logistic regression coefficient values > 0

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(model_df[cols_to_keep], model_df['class'], test_size=0.3, random_state=42)

In [ ]:
lrc2 = LogisticRegression()
lrc2.fit(X_train2, y_train2)
y_pred_lr2 = lrc2.predict(X_test2)

#### Confusion Matrix

In [ ]:
sns.heatmap(confusion_matrix(y_test2, y_pred_lr2), annot=True)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.show()

#### Accuracy Score

In [ ]:
lrc2.score(X_test2, y_test2)

#### Classification Report

In [ ]:
print(classification_report(y_test2, y_pred_lr2))

It appears from the above that filtering the feature set based on coefficient > 0 didn't help improve the performance of the logistic regression model at all.

### Building a Random Forest Ensemble Classifier

As we can see below, the random forest ensemble performs much better than logistic regression or gaussian naive bayes.  Compared to the extra trees ensemble below, though, they perform very similarly, there is only 0.002 difference in accuracy, and the confusion matrices look almost the same depending on training and run version.  The major difference is that random forest takes a little longer to train, so performance wise, the extra trees is better for production.

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred_rf = rfc.predict(X_test)

#### Confusion Matrix

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.show()

In [ ]:
rfc.score(X_test, y_test)

#### Classification Report

In [ ]:
print(classification_report(y_test, y_pred_rf, labels=[False, True]))

### Building an Extra Trees Ensemble Classifier

As we will see below, the extra trees classifier easily out performs the other vanilla classifiers without any kin dof hyperparameter tuning of the models.  This ensemble achieves near 100% accuracy, and very minimal Type I or Type II errors.  However, we are both skeptic that this is overfitting on multicolliearity of sequence length and molecular weight, as well as potential bias or limitations in the dataset downloaded from the NCBI Virus database.

In [ ]:
etc = ExtraTreesClassifier()
etc.fit(X_train, y_train)
y_pred_et = etc.predict(X_test)

#### Confusion Matrix

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred_et), annot=True)
plt.xlabel('predicted')
plt.ylabel('actual')
plt.show()

Here we can see there are only 8 false positives and 12 false negatives.  Very highly performant model!

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(etc, X_test, y_test)  # doctest: +SKIP
plt.grid(None)
plt.show()

#### Near 100% accuracy score

In [ ]:
etc.score(X_test, y_test)

#### Classification Report

In [ ]:
print(classification_report(y_test, y_pred_et, labels=[False, True]))

#### Estimating what the most important features are

In [ ]:
est = etc.estimators_
importance = etc.feature_importances_
importance

In [ ]:
X_train.columns

In [ ]:
indices = np.argsort(importance)[::-1]

In [ ]:
for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importance[indices[f]]))

In [ ]:
indices

In [ ]:
importance

In [ ]:
train_cols = []

In [ ]:
for ii in indices:
    train_cols.append(X_train.columns[ii])

In [ ]:
imp_df = pd.DataFrame({
    'index': indices,
    'feature': X_train.columns.tolist(),
    'importance': importance
})

In [ ]:
imp_df.sort_values(by='importance', ascending=True, inplace=True)

In [ ]:
imp_df.plot.bar(x='feature', y='importance', rot=90, figsize=(10,5))
plt.title('Feature Importance (Extra Trees Classifier)');

In [ ]:
importance = etc.feature_importances_
feats = {}
# summarize feature importance
for i,v in zip(X_train.columns, importance):
    feats[i] = v
    print(i,'Score: %.5f' % (v))
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance').plot(kind='bar', rot=90, figsize=(10,10), title = "Feature Importance (Extra Tree Classifier)")

In [ ]:
from sklearn.model_selection import learning_curve
def plot_curve(lg, X,y):
    # instantiate

    # fit
    
    
    """
    Generate a simple plot of the test and traning learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : integer, cross-validation generator, optional
        If an integer is passed, it is the number of folds (defaults to 3).
        Specific cross-validation objects can be passed, see
        sklearn.cross_validation module for the list of possible objects

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
        
    x1 = np.linspace(0, 10, 8, endpoint=True) produces
        8 evenly spaced points in the range 0 to 10
    """
    
    train_sizes, train_scores, test_scores = learning_curve(lg, X, y, cv = 5,n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5), verbose=0)

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.figure()
    plt.title("ExtraTreeClassifier")
    plt.legend(loc="best")
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.gca().invert_yaxis()
    
    # box-like grid
    plt.grid()
    
    # plot the std deviation as a transparent range at each training set size
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    
    # plot the average training and test score lines at each training set size
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    
    # sizes the window for readability and displays the plot
    # shows error from 0 to 1.1
    plt.ylim(-.1,1.1)
    plt.show()

In [ ]:
plot_curve(etc,X_train,y_train)

In [ ]:
#A precision-recall curve shows the relationship between precision (= positive predictive value) and recall (= sensitivity) for every possible cut-off.
from sklearn.metrics import plot_precision_recall_curve

disp = plot_precision_recall_curve(etc, X_test, y_test)

In [ ]:
length = X_test['seq_len']
weights = X_test['molecular_weights']
gc = X_test['gc_content']
eaa = X_test['amino_acid_E']
paa = X_test['amino_acid_P']
actual = y_test.values
predicted = y_pred

In [ ]:
model_out = pd.DataFrame({
    'length': length,
    'weight': weights,
    'gc_content': gc,
    'amino_acid_e': eaa,
    'amino_acid_p': paa,
    'actual': actual,
    'predicted': predicted
    })

In [ ]:
for col in ['weight','gc_content','amino_acid_e','amino_acid_p']:
    g = sns.FacetGrid(model_out, col='actual',  hue="predicted", hue_order=[True, False])
    g = (g.map(plt.scatter, 'length', col, edgecolor="w")
          .add_legend())
    plt.title(f'Comparison of Truth to Prediction on Length to {col}\n')

In [ ]:
fig = px.scatter_3d(model_out, x="weight", y="amino_acid_e", z="gc_content", color="predicted", opacity=.25,
                 hover_data=['length','weight','gc_content','amino_acid_e','actual','predicted'], color_continuous_scale=px.colors.sequential.Viridis)
fig.show()

#### Visualization of Extra Trees Classifier Decision Tree

From the full decision tree diagram below for the Extra Trees Classifier, it's difficult to make out the decisions at nodes at this resolution, but if you view the .PNG output file, you can scroll around to see how the model is making decisions at each node.  The first split is at Amino Acid H, then the next tier splits at Molecular Weights, and Amino Acid F, and so on down the tree.

In [ ]:
fn=['seq_len', 'gc_content', 'molecular_weights', 'aromaticity',
       'instability_index', 'amino_acid_A', 'amino_acid_C', 'amino_acid_D',
       'amino_acid_E', 'amino_acid_F', 'amino_acid_G', 'amino_acid_H',
       'amino_acid_I', 'amino_acid_K', 'amino_acid_L', 'amino_acid_M',
       'amino_acid_N', 'amino_acid_P', 'amino_acid_Q', 'amino_acid_R',
       'amino_acid_S', 'amino_acid_T', 'amino_acid_V', 'amino_acid_W',
       'amino_acid_Y']
cn=['virulent','non-virulent']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (20,20), dpi=800)
tree.plot_tree(etc.estimators_[0],
               feature_names = fn,
               class_names=cn,
               filled = True);
fig.savefig('individualtree.png')

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
#Predict probabilities for the test data.
y_proba = etc.predict_proba(X_test)
#Keep Probabilities of the positive class only.
y_proba = y_proba[:, 1]
#Compute the AUC Score.
auc = roc_auc_score(y_test, y_proba)
print('AUC: %.2f' % auc)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_proba, pos_label=1)
plot_roc_curve(fpr, tpr)

### Evaluate Model Performance with AUROC

In [ ]:
def plot_roc(X, y, models, model_names):
    plt.figure(0, figsize = [8, 7]).clf()
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([-.01, 1.01])
    plt.ylim([-.01, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    
    for ii, model in enumerate(models):
        y_prob = model.predict_proba(X)[:, 1]
        fpr, tpr, threshold = roc_curve(y, y_prob)
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, label = "{} AUC = {:0.2f}".format(model_names[ii], roc_auc))

    plt.legend(loc = 'lower right');

In [ ]:
plot_roc(X_test, y_test, [etc, rfc, lrc, gnb], ['extra trees', 'random forest', 'logistic regression','gaussian naive bayes'])

In conclusion, we achieved some interesting results with clustering tools, but ultimately some of the simplest features obtained from the raw data and protein analysis yielded the best predictive results if a virus is virulent or not.  Clearly the ensemble Extra Trees model worked the best of the three models attempted here.  Perhaps others would have performed just as well or in between the gap but achieving near 100% accuracy and minimal Type I and Type II errors tells us that the Extra Trees ensemble is ready for deployment.

### Writing ExtraTreesClassifier model to disk

In [ ]:
import joblib
joblib.dump(etc, 'virus_extra_trees_model.joblib')